In [ ]:
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")

In [ ]:
llm = ChatGroq(model="gemma2-9b-it", temperature=0)

In [ ]:
def reverse_string(s):
    """Reverses the input string."""
    return s[::-1]

def capitalize_string(s):
    """Capsitizes the input string."""
    return s[:1].upper() + s[1:]

def to_lowercase(s):
    """Converts the input string to lowercase."""
    return s.lower()

def to_uppercase(s):
    """Converts the input string to uppercase."""
    return s.upper()


In [ ]:
tools = [reverse_string, capitalize_string, to_lowercase, to_uppercase]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
sys_msg = SystemMessage(content="Your are a helpful assistant tasked with performing string functions")


In [ ]:
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

#### Build Agent

In [ ]:
# Graph
builder = StateGraph(MessagesState)
memory = MemorySaver()

# Define nodees: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edgesL these determine how the flow moves

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition,)

builder.add_edge("tools", "assistant")

react_graph = builder.compile(checkpointer=memory)


In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
messages = [HumanMessage(content="Reverse this string 'Charles McCullough'")]
messages = react_graph.invoke({"messages": messages}, config)

for m in messages['messages']:
    m.pretty_print()


In [ ]:
messages = [HumanMessage(content="Now convert the reversed string to upper case.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="Now reverse the string again and convert it to lowercase.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="Now capitize the string.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

In [ ]:
print(capitalize_string("charles mccullough"))